In [2]:
#Script for scrape post and replies from file prepared in fico_forum_scraper_ipynb
from bs4 import BeautifulSoup
import requests
import re
from lxml import html
from datetime import datetime
from itertools import chain
import pandas as pd
import unicodedata
from tqdm import tqdm
from os import path
from os import listdir


In [2]:
posts_df=pd.read_csv('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/posts_to_scrape.csv')

In [3]:
posts_df

,post_id,post_date,post_title,post_author,post_replies,post_link
0,6619712,2022-11-30,Garden Club - December 2022,UncleB,535,https://ficoforums.myfico.com/t5/Personal-Fina...
1,6624791,2022-12-16,The 42 billion dollar question,AndySoCal,3,https://ficoforums.myfico.com/t5/Personal-Fina...
2,6626687,2022-12-23,Navy Federal Internal Score,tebtengri,5,https://ficoforums.myfico.com/t5/Personal-Fina...
3,6604324,2022-10-16,high yield savings accounts,acreditdummy123,20,https://ficoforums.myfico.com/t5/Personal-Fina...
4,6612558,2022-11-07,Does the Fold debit card check ChexSystems?,TyrannicalDuncery,6,https://ficoforums.myfico.com/t5/Personal-Fina...
...,...,...,...,...,...,...
482,6468010,2022-01-04,New blog article: How Minimalism Can Help You ...,Elizabeth_FICO,2,https://ficoforums.myfico.com/t5/Personal-Fina...
483,6466879,2022-01-01,Tucson Federal Credit Union HP for regular sav...,SouthJamaica,2,https://ficoforums.myfico.com/t5/Personal-Fina...
484,6466656,2022-01-01,PENFED Login,outofcredit,3,https://ficoforums.myfico.com/t5/Personal-Fina...
485,6381660,2022-04-11,Charles Schwab Investor Checking,UncleB,14,https://ficoforums.myfico.com/t5/Personal-Fina...


In [4]:

def get_num_pages_post(url):
    last_page=''
    num_pages=[]
    content=requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")
    pages=soup.find_all('ul', {'class':'lia-paging-full-pages'})
    if len(pages)>0:
        for page in pages[:1]:
            last_page=soup.find('li', {'class':re.compile('lia-paging-page-last')}).text
        num_pages=list(range(2,int(last_page)+1))
    
    return num_pages

In [5]:
def create_url_pages_post(url):
    pages_url=[url]
    pages_num=get_num_pages_post(url)
    for p in pages_num:
        p_url=f'{url}/page/{str(p)}'
        pages_url.append(p_url)
    return(pages_url)

In [6]:
def get_reply_date(reply_cell):
    post_date=''
    try:
        post_date=reply_cell.find('span', {'class':'local-date'}).text.strip()
        post_date=post_date.encode('ascii','ignore')
        post_date=post_date.decode()
        post_date=datetime.strptime(post_date, '%m-%d-%Y')
    except AttributeError:
        pass
    return post_date

In [7]:
def get_reply_author(reply_cell):
    replier=''
    try:
        replier=reply_cell.find('span', {'class':'login-bold'}).text.strip()
    except AttributeError:
        pass
    return replier

In [8]:
def get_post_text(url):
    text=''
    content=requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")
    post_cell=soup.find('div', {'class':'lia-component-topic-message'})
    text_cell=post_cell.find('div', {'class':'lia-message-body-content'})
    signature_cell=text_cell.find('div', {'class': re.compile('UserSignature')})
    if signature_cell!=None:
        signature_cell.extract()
        text=text_cell.text
        #text=text.encode('ascii','ignore')
        #text=text.decode()
        text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
    
    else:
        text=text_cell.text
        #text=text.encode('ascii','ignore')
        #text=text.decode()
        text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
    
    return text


    


In [9]:
def get_post_author_rank(url):
    rank=''
    content=requests.get(url).text
    soup = BeautifulSoup(content, "html.parser")
    try:

        post_cell=soup.find('div', {'class':'lia-component-topic-message'})
        rank=post_cell.find('div', {'class':re.compile('lia-message-author-rank')}).text.strip()
    except AttributeError:
        pass
    return rank
    


In [10]:
def get_reply_text(reply_cell):
    text=''
    text_cell=reply_cell.find('div', {'class':'lia-message-body-content'})
    signature_cell=text_cell.find('div', {'class': re.compile('UserSignature')})
    blockquote_cell=text_cell.find('blockquote')
    if signature_cell!=None:
        signature_cell.extract()
        text=text_cell.text
        #text=text.encode('ascii','ignore')
        #text=text.decode()
        text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
        if blockquote_cell!=None:
            blockquote_cell.extract()
            text=text_cell.text
            #text=text.encode('ascii','ignore')
            #text=text.decode()
            text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
    
    else:
        text=text_cell.text
        #text=text.encode('ascii','ignore')
        #text=text.decode()
        text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
        if blockquote_cell!=None:
            blockquote_cell.extract()
            text=text_cell.text
            #text=text.encode('ascii','ignore')
            #text=text.decode()
            text=unicodedata.normalize('NFKD', text).replace('\n',' ').strip()
    
    return text


In [11]:
def get_votes(reply_cell):
    votes=''
    try:
        votes=reply_cell.find('span', {'itemprop':'upvoteCount'}).text.strip()
        votes=int(votes)
    except AttributeError:
        pass
       
    return votes

In [12]:
def get_replier_rank(reply_cell):
    rank=''
    try:
        rank=reply_cell.find('div', {'class':re.compile('lia-message-author-rank')}).text.strip()
    except AttributeError:
        pass
    return rank

In [13]:
def scrape_discussion(df,row):
    discussion_urls=create_url_pages_post(df['post_link'].iloc[row])
    post_id=df['post_id'].iloc[row]
    post_date=df['post_date'].iloc[row]
    post_title=df['post_title'].iloc[row]
    post_author=df['post_author'].iloc[row]
    post_text=get_post_text(discussion_urls[0])
    post_author_rank=get_post_author_rank(discussion_urls[0])
    discussion_data=[]
    for page in discussion_urls:
        
        page_data=[]
        date=None
        replier=None
        votes=None
        rank=None
        text=None
        content=requests.get(page).text
        soup = BeautifulSoup(content, "html.parser")
        replies_list=soup.find('div', {'class':'lia-component-reply-list'})
        replies=replies_list.find_all('div', {'class':'lia-linear-display-message-view'})
        for reply in replies:
             date=get_reply_date(reply)
             replier=get_reply_author(reply)
             votes=get_votes(reply)
             rank=get_replier_rank(reply)
             text=get_reply_text(reply)
             discussion_data.append((post_id, post_date, post_title, post_author, post_text, post_author_rank, date, replier, votes, rank, text))
    posts_df=pd.DataFrame(discussion_data, columns=['post_id', 'post_date', 'post_title', 'post_author', 'post_text', 'post_author_rank', 'reply_date', 'replier', 'votes', 'replier_rank', 'reply_text'])
    posts_df.to_csv(f'/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/ficoforum_data/{post_id}.csv')
    

        
            
        

    
    

In [14]:
for i in tqdm(range(len(posts_df))):
    scrape_discussion(posts_df,i)

100%|██████████| 487/487 [43:32<00:00,  5.36s/it]  


In [6]:
all_files=listdir('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/ficoforum_data/')


main_paths=['/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/ficoforum_data/']
all_paths_data=[]
for path_m in main_paths:
    
    all_files=listdir(path_m)
    for p in all_files:
        if '~lock' in p:
            continue
        file_path=str(path_m)+str(p)
        all_paths_data.append(file_path)

In [7]:
fico_df=pd.concat((pd.read_csv(f,index_col=0) for f in all_paths_data), ignore_index=True )

In [9]:
fico_df.to_csv('/home/slawomir/Portfolio/omdena_nlp_finance_chatbot/fico_forum_all_data.csv', index=False)